In [432]:
import numpy as np
import pandas as pd
from sportsreference.ncaab.teams import Teams
from sportsreference.ncaab.teams import Team
from tqdm import tqdm
from sportsreference.ncaab.boxscore2 import Boxscore
import time
from sportsreference.ncaab.roster import Player
from kenpompy.utils import login
import kenpompy.summary as kp

In [171]:
seasons = [2021]
master_array = []
for season in seasons: 
    try:
        season_team_list = Teams(str(season))
        for team in season_team_list:
            print(team, end = '\r')
            team_schedule = team.schedule
            for game in team_schedule:
                boxscore = game.boxscore_index
                master_array.append([str(season),team.name,boxscore])
    except:
        continue
            
schedule_df = pd.DataFrame(master_array, columns=['Season','TeamName','BoxscoreIndex'])
schedule_df.to_csv('2021_schedule.csv')
schedule_df

The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun

,Season,TeamName,BoxscoreIndex
0,2021,Abilene Christian,2020-11-25-11-abilene-christian
1,2021,Abilene Christian,2020-11-27-14-austin-peay
2,2021,Abilene Christian,2020-11-28-11-nebraska-omaha
3,2021,Abilene Christian,2020-12-01-20-abilene-christian
4,2021,Abilene Christian,2020-12-05-20-abilene-christian
...,...,...,...
8226,2021,Youngstown State,2021-02-13-17-youngstown-state
8227,2021,Youngstown State,2021-02-19-17-youngstown-state
8228,2021,Youngstown State,2021-02-20-17-youngstown-state
8229,2021,Youngstown State,2021-02-25-20-youngstown-state


In [102]:
seasons = [2014,2015,2016,2017,2018,2019,2020]
for season in seasons: 
# will run once for each season
    full_box_df = None
    counter = 0
    season_df = schedule_df.loc[schedule_df.Season==str(season)]
    for index, row in tqdm(season_df.iterrows()):
        if counter < 500:
            box_link = row['BoxscoreIndex']
            try:
                _df = Boxscore(box_link).dataframe
                _df['season'] = season

                if full_box_df is not None:
                    box_df = pd.concat([box_df,_df],axis=0)
                    counter += 1
                else:
                    full_box_df = _df
                    counter +=1
            except:
                counter += 1
                pass
        else:
            time.sleep(np.random.randint(40,90))
            counter = 0
box_df.to_csv('2014_2015_2016_2017_2018_2019_2020_2021_games_final.csv')

8231it [48:24,  2.83it/s]


The HTML changed for 2021, so I had to edit the .py file to accomodate and run 2021 separately

In [180]:
from sportsreference.ncaab.boxscore2 import Boxscore
seasons = [2021]
for season in seasons: 
    box_df_21 = None
    counter = 0
    season_df = schedule_df.loc[schedule_df.Season==str(season)]
    for index, row in tqdm(season_df.iterrows()):
        if counter < 500:
            box_link = row['BoxscoreIndex']
            try:
                _df = Boxscore(box_link).dataframe
                _df['season'] = season

                if full_box_df is not None:
                    box_df_21 = pd.concat([box_df_21,_df],axis=0)
                    counter += 1
                else:
                    box_df_21 = _df
                    counter +=1
            except:
                counter += 1
                pass
        else:
            time.sleep(np.random.randint(40,90))
            counter = 0
box_df_21.to_csv('2021_games_final.csv')

8231it [2:50:23,  1.24s/it]


In [183]:
box_df = pd.concat([box_df,box_df_21])

In [184]:
pd.set_option('display.max_columns', None)
box_df['losing_name'] = box_df['losing_name'].str.replace("\n",'',regex = True)
box_df['losing_name'] = box_df['losing_name'].str.replace("\t",'',regex = True)
box_df.replace('\n','',regex = True,inplace = True)
box_df.replace('\t','',regex = True,inplace = True)
box_df.replace('&amp','',regex = True,inplace = True)
box_df['home_ranking'] = box_df['home_ranking'].fillna(0)
box_df['away_ranking'] = box_df['away_ranking'].fillna(0)
len(box_df)

86217

In [185]:
box_df_ = box_df.copy()
box_df = box_df.dropna()
len(box_df)

86164

In [186]:
box_df.to_csv('full_box_df')

In [187]:


list_of_stats = ['team','home','opponent','date','season','pace','result','wins','win_percentage','two_point_field_goals','two_point_field_goal_percentage','two_point_field_goal_attempts','turnovers','turnover_percentage',
            'true_shooting_percentage','total_rebounds','total_rebound_percentage','three_point_field_goals','three_point_field_goal_percentage','three_point_field_goal_attempts',
            'three_point_attempt_rate','steals','steal_percentage','ranking','points','personal_fouls','offensive_rebounds','offensive_rebound_percentage','offensive_rating','minutes_played',
            'losses','free_throws','free_throw_percentage','free_throw_attempts','free_throw_attempt_rate','field_goals','field_goal_percentage','field_goal_attempts','effective_field_goal_percentage',
            'defensive_rebounds','defensive_rating','blocks','block_percentage','assists','assist_percentage']

for stat in list_of_stats[7:]:
    list_of_stats.append('opponent_'+stat)
    
stats_dict = {key:[] for key in list_of_stats}

for Team in tqdm(box_df.losing_name.unique().tolist()):

    #Home/Winner
    temp_df = box_df[(box_df.winning_name == Team) & (box_df.winner == "Home")]
    stats_dict['team'] += temp_df.winning_name.tolist()
    stats_dict['opponent']  += temp_df.losing_name.tolist()
    stats_dict['date']  += temp_df.date.tolist()
    stats_dict['pace']  += temp_df.pace.tolist()
    stats_dict['wins']  += temp_df.home_wins.tolist()
    stats_dict['win_percentage']  += temp_df.home_win_percentage.tolist()
    stats_dict['two_point_field_goals']  += temp_df.home_two_point_field_goals.tolist()
    stats_dict['two_point_field_goal_percentage']  += temp_df.home_two_point_field_goal_percentage.tolist() 
    stats_dict['two_point_field_goal_attempts']  += temp_df.home_two_point_field_goal_attempts.tolist()
    stats_dict['turnovers']  += temp_df.home_turnovers.tolist()
    stats_dict['turnover_percentage']  += temp_df.home_turnover_percentage.tolist()
    stats_dict['true_shooting_percentage']  += temp_df.home_true_shooting_percentage.tolist()
    stats_dict['total_rebounds']  += temp_df.home_total_rebounds.tolist()
    stats_dict['total_rebound_percentage']  += temp_df.home_total_rebound_percentage.tolist()
    stats_dict['three_point_field_goals']  += temp_df.home_three_point_field_goals.tolist()
    stats_dict['three_point_field_goal_percentage']  += temp_df.home_three_point_field_goal_percentage.tolist()
    stats_dict['three_point_field_goal_attempts']  += temp_df.home_three_point_field_goal_attempts.tolist()
    stats_dict['three_point_attempt_rate']  += temp_df.home_three_point_attempt_rate.tolist()
    stats_dict['steals']  += temp_df.home_steals.tolist()
    stats_dict['steal_percentage']  += temp_df.home_steal_percentage.tolist()
    stats_dict['ranking']  += temp_df.home_ranking.tolist()
    stats_dict['points']  += temp_df.home_points.tolist()
    stats_dict['personal_fouls']  += temp_df.home_personal_fouls.tolist()
    stats_dict['offensive_rebounds']  += temp_df.home_offensive_rebounds.tolist()
    stats_dict['offensive_rebound_percentage']  += temp_df.home_offensive_rebound_percentage.tolist()
    stats_dict['offensive_rating']  += temp_df.home_offensive_rating.tolist()
    stats_dict['minutes_played']  += temp_df.home_minutes_played.tolist()
    stats_dict['losses']  += temp_df.home_losses.tolist()
    stats_dict['free_throws']  += temp_df.home_free_throws.tolist()
    stats_dict['free_throw_percentage']  += temp_df.home_free_throw_percentage.tolist()
    stats_dict['free_throw_attempts']  += temp_df.home_free_throw_attempts.tolist()
    stats_dict['free_throw_attempt_rate']  += temp_df.home_free_throw_attempt_rate.tolist()
    stats_dict['field_goals']  += temp_df.home_field_goals.tolist()
    stats_dict['field_goal_percentage']  += temp_df.home_field_goal_percentage.tolist()
    stats_dict['field_goal_attempts']  += temp_df.home_field_goal_attempts.tolist()
    stats_dict['effective_field_goal_percentage']  += temp_df.home_effective_field_goal_percentage.tolist()
    stats_dict['defensive_rebounds']  += temp_df.home_defensive_rebounds.tolist()
    stats_dict['defensive_rating']  += temp_df.home_free_throws.tolist()
    stats_dict['blocks']  += temp_df.home_blocks.tolist()
    stats_dict['block_percentage']  += temp_df.home_block_percentage.tolist()
    stats_dict['assists']  += temp_df.home_assists.tolist()
    stats_dict['assist_percentage']  += temp_df.home_assist_percentage.tolist()
    stats_dict['result'] += [1] * len(temp_df)
    stats_dict['opponent_wins'] += temp_df.away_wins.tolist()
    stats_dict['opponent_win_percentage']  += temp_df.away_win_percentage.tolist()
    stats_dict['opponent_two_point_field_goals']  += temp_df.away_two_point_field_goals.tolist()
    stats_dict['opponent_two_point_field_goal_percentage']  += temp_df.away_two_point_field_goal_percentage.tolist() 
    stats_dict['opponent_two_point_field_goal_attempts']  += temp_df.away_two_point_field_goal_attempts.tolist()
    stats_dict['opponent_turnovers']  += temp_df.away_turnovers.tolist()
    stats_dict['opponent_turnover_percentage']  += temp_df.away_turnover_percentage.tolist()
    stats_dict['opponent_true_shooting_percentage']  += temp_df.away_true_shooting_percentage.tolist()
    stats_dict['opponent_total_rebounds']  += temp_df.away_total_rebounds.tolist()
    stats_dict['opponent_total_rebound_percentage']  += temp_df.away_total_rebound_percentage.tolist()
    stats_dict['opponent_three_point_field_goals']  += temp_df.away_three_point_field_goals.tolist()
    stats_dict['opponent_three_point_field_goal_percentage']  += temp_df.away_three_point_field_goal_percentage.tolist()
    stats_dict['opponent_three_point_field_goal_attempts']  += temp_df.away_three_point_field_goal_attempts.tolist()
    stats_dict['opponent_three_point_attempt_rate']  += temp_df.away_three_point_attempt_rate.tolist()
    stats_dict['opponent_steals']  += temp_df.away_steals.tolist()
    stats_dict['opponent_steal_percentage']  += temp_df.away_steal_percentage.tolist()
    stats_dict['opponent_ranking']  += temp_df.away_ranking.tolist()
    stats_dict['opponent_points']  += temp_df.away_points.tolist()
    stats_dict['opponent_personal_fouls']  += temp_df.away_personal_fouls.tolist()
    stats_dict['opponent_offensive_rebounds']  += temp_df.away_offensive_rebounds.tolist()
    stats_dict['opponent_offensive_rebound_percentage']  += temp_df.away_offensive_rebound_percentage.tolist()
    stats_dict['opponent_offensive_rating']  += temp_df.away_offensive_rating.tolist()
    stats_dict['opponent_minutes_played']  += temp_df.away_minutes_played.tolist()
    stats_dict['opponent_losses']  += temp_df.away_losses.tolist()
    stats_dict['opponent_free_throws']  += temp_df.away_free_throws.tolist()
    stats_dict['opponent_free_throw_percentage']  += temp_df.away_free_throw_percentage.tolist()
    stats_dict['opponent_free_throw_attempts']  += temp_df.away_free_throw_attempts.tolist()
    stats_dict['opponent_free_throw_attempt_rate']  += temp_df.away_free_throw_attempt_rate.tolist()
    stats_dict['opponent_field_goals']  += temp_df.away_field_goals.tolist()
    stats_dict['opponent_field_goal_percentage']  += temp_df.away_field_goal_percentage.tolist()
    stats_dict['opponent_field_goal_attempts']  += temp_df.away_field_goal_attempts.tolist()
    stats_dict['opponent_effective_field_goal_percentage']  += temp_df.away_effective_field_goal_percentage.tolist()
    stats_dict['opponent_defensive_rebounds']  += temp_df.away_defensive_rebounds.tolist()
    stats_dict['opponent_defensive_rating']  += temp_df.away_free_throws.tolist()
    stats_dict['opponent_blocks']  += temp_df.away_blocks.tolist()
    stats_dict['opponent_block_percentage']  += temp_df.away_block_percentage.tolist()
    stats_dict['opponent_assists']  += temp_df.away_assists.tolist()
    stats_dict['opponent_assist_percentage']  += temp_df.away_assist_percentage.tolist()
    stats_dict['home'] += [1] * len(temp_df)
    stats_dict['season']  += temp_df.season.tolist()

    #Away/Winner  Games won on the road
    temp_df = box_df[(box_df.winning_name == Team) & (box_df.winner == "Away")]
    stats_dict['team']  += temp_df.winning_name.tolist()
    stats_dict['opponent']  += temp_df.losing_name.tolist()
    stats_dict['date']  += temp_df.date.tolist()
    stats_dict['pace']  += temp_df.pace.tolist()
    stats_dict['wins']  += temp_df.away_wins.tolist()
    stats_dict['win_percentage']  += temp_df.away_win_percentage.tolist()
    stats_dict['two_point_field_goals']  += temp_df.away_two_point_field_goals.tolist()
    stats_dict['two_point_field_goal_percentage']  += temp_df.away_two_point_field_goal_percentage.tolist() 
    stats_dict['two_point_field_goal_attempts']  += temp_df.away_two_point_field_goal_attempts.tolist()
    stats_dict['turnovers']  += temp_df.away_turnovers.tolist()
    stats_dict['turnover_percentage']  += temp_df.away_turnover_percentage.tolist()
    stats_dict['true_shooting_percentage']  += temp_df.away_true_shooting_percentage.tolist()
    stats_dict['total_rebounds']  += temp_df.away_total_rebounds.tolist()
    stats_dict['total_rebound_percentage']  += temp_df.away_total_rebound_percentage.tolist()
    stats_dict['three_point_field_goals']  += temp_df.away_three_point_field_goals.tolist()
    stats_dict['three_point_field_goal_percentage']  += temp_df.away_three_point_field_goal_percentage.tolist()
    stats_dict['three_point_field_goal_attempts']  += temp_df.away_three_point_field_goal_attempts.tolist()
    stats_dict['three_point_attempt_rate']  += temp_df.away_three_point_attempt_rate.tolist()
    stats_dict['steals']  += temp_df.away_steals.tolist()
    stats_dict['steal_percentage']  += temp_df.away_steal_percentage.tolist()
    stats_dict['ranking']  += temp_df.away_ranking.tolist()
    stats_dict['points']  += temp_df.away_points.tolist()
    stats_dict['personal_fouls']  += temp_df.away_personal_fouls.tolist()
    stats_dict['offensive_rebounds']  += temp_df.away_offensive_rebounds.tolist()
    stats_dict['offensive_rebound_percentage']  += temp_df.away_offensive_rebound_percentage.tolist()
    stats_dict['offensive_rating']  += temp_df.away_offensive_rating.tolist()
    stats_dict['minutes_played']  += temp_df.away_minutes_played.tolist()
    stats_dict['losses']  += temp_df.away_losses.tolist()
    stats_dict['free_throws']  += temp_df.away_free_throws.tolist()
    stats_dict['free_throw_percentage']  += temp_df.away_free_throw_percentage.tolist()
    stats_dict['free_throw_attempts']  += temp_df.away_free_throw_attempts.tolist()
    stats_dict['free_throw_attempt_rate']  += temp_df.away_free_throw_attempt_rate.tolist()
    stats_dict['field_goals']  += temp_df.away_field_goals.tolist()
    stats_dict['field_goal_percentage']  += temp_df.away_field_goal_percentage.tolist()
    stats_dict['field_goal_attempts']  += temp_df.away_field_goal_attempts.tolist()
    stats_dict['effective_field_goal_percentage']  += temp_df.away_effective_field_goal_percentage.tolist()
    stats_dict['defensive_rebounds']  += temp_df.away_defensive_rebounds.tolist()
    stats_dict['defensive_rating']  += temp_df.away_free_throws.tolist()
    stats_dict['blocks']  += temp_df.away_blocks.tolist()
    stats_dict['block_percentage']  += temp_df.away_block_percentage.tolist()
    stats_dict['assists']  += temp_df.away_assists.tolist()
    stats_dict['assist_percentage']  += temp_df.away_assist_percentage.tolist()
    stats_dict['result'] += [1] * len(temp_df)
    stats_dict['opponent_wins']  += temp_df.home_wins.tolist()
    stats_dict['opponent_win_percentage']  += temp_df.home_win_percentage.tolist()
    stats_dict['opponent_two_point_field_goals']  += temp_df.home_two_point_field_goals.tolist()
    stats_dict['opponent_two_point_field_goal_percentage']  += temp_df.home_two_point_field_goal_percentage.tolist() 
    stats_dict['opponent_two_point_field_goal_attempts']  += temp_df.home_two_point_field_goal_attempts.tolist()
    stats_dict['opponent_turnovers']  += temp_df.home_turnovers.tolist()
    stats_dict['opponent_turnover_percentage']  += temp_df.home_turnover_percentage.tolist()
    stats_dict['opponent_true_shooting_percentage']  += temp_df.home_true_shooting_percentage.tolist()
    stats_dict['opponent_total_rebounds']  += temp_df.home_total_rebounds.tolist()
    stats_dict['opponent_total_rebound_percentage']  += temp_df.home_total_rebound_percentage.tolist()
    stats_dict['opponent_three_point_field_goals']  += temp_df.home_three_point_field_goals.tolist()
    stats_dict['opponent_three_point_field_goal_percentage']  += temp_df.home_three_point_field_goal_percentage.tolist()
    stats_dict['opponent_three_point_field_goal_attempts']  += temp_df.home_three_point_field_goal_attempts.tolist()
    stats_dict['opponent_three_point_attempt_rate']  += temp_df.home_three_point_attempt_rate.tolist()
    stats_dict['opponent_steals']  += temp_df.home_steals.tolist()
    stats_dict['opponent_steal_percentage']  += temp_df.home_steal_percentage.tolist()
    stats_dict['opponent_ranking']  += temp_df.home_ranking.tolist()
    stats_dict['opponent_points']  += temp_df.home_points.tolist()
    stats_dict['opponent_personal_fouls']  += temp_df.home_personal_fouls.tolist()
    stats_dict['opponent_offensive_rebounds']  += temp_df.home_offensive_rebounds.tolist()
    stats_dict['opponent_offensive_rebound_percentage']  += temp_df.home_offensive_rebound_percentage.tolist()
    stats_dict['opponent_offensive_rating']  += temp_df.home_offensive_rating.tolist()
    stats_dict['opponent_minutes_played']  += temp_df.home_minutes_played.tolist()
    stats_dict['opponent_losses']  += temp_df.home_losses.tolist()
    stats_dict['opponent_free_throws']  += temp_df.home_free_throws.tolist()
    stats_dict['opponent_free_throw_percentage']  += temp_df.home_free_throw_percentage.tolist()
    stats_dict['opponent_free_throw_attempts']  += temp_df.home_free_throw_attempts.tolist()
    stats_dict['opponent_free_throw_attempt_rate']  += temp_df.home_free_throw_attempt_rate.tolist()
    stats_dict['opponent_field_goals']  += temp_df.home_field_goals.tolist()
    stats_dict['opponent_field_goal_percentage']  += temp_df.home_field_goal_percentage.tolist()
    stats_dict['opponent_field_goal_attempts']  += temp_df.home_field_goal_attempts.tolist()
    stats_dict['opponent_effective_field_goal_percentage']  += temp_df.home_effective_field_goal_percentage.tolist()
    stats_dict['opponent_defensive_rebounds']  += temp_df.home_defensive_rebounds.tolist()
    stats_dict['opponent_defensive_rating']  += temp_df.home_free_throws.tolist()
    stats_dict['opponent_blocks']  += temp_df.home_blocks.tolist()
    stats_dict['opponent_block_percentage']  += temp_df.home_block_percentage.tolist()
    stats_dict['opponent_assists']  += temp_df.home_assists.tolist()
    stats_dict['opponent_assist_percentage']  += temp_df.home_assist_percentage.tolist()
    stats_dict['home'] += [0] * len(temp_df)
    stats_dict['season']  += temp_df.season.tolist()

    #Away/loser Games lost at home
    temp_df = box_df[(box_df.losing_name == Team) & (box_df.winner == "Away")]
    stats_dict['team']  += temp_df.losing_name.tolist()
    stats_dict['opponent']  += temp_df.winning_name.tolist()
    stats_dict['date']  += temp_df.date.tolist()
    stats_dict['pace']  += temp_df.pace.tolist()
    stats_dict['wins']  += temp_df.home_wins.tolist()
    stats_dict['win_percentage']  += temp_df.home_win_percentage.tolist()
    stats_dict['two_point_field_goals']  += temp_df.home_two_point_field_goals.tolist()
    stats_dict['two_point_field_goal_percentage']  += temp_df.home_two_point_field_goal_percentage.tolist() 
    stats_dict['two_point_field_goal_attempts']  += temp_df.home_two_point_field_goal_attempts.tolist()
    stats_dict['turnovers']  += temp_df.home_turnovers.tolist()
    stats_dict['turnover_percentage']  += temp_df.home_turnover_percentage.tolist()
    stats_dict['true_shooting_percentage']  += temp_df.home_true_shooting_percentage.tolist()
    stats_dict['total_rebounds']  += temp_df.home_total_rebounds.tolist()
    stats_dict['total_rebound_percentage']  += temp_df.home_total_rebound_percentage.tolist()
    stats_dict['three_point_field_goals']  += temp_df.home_three_point_field_goals.tolist()
    stats_dict['three_point_field_goal_percentage']  += temp_df.home_three_point_field_goal_percentage.tolist()
    stats_dict['three_point_field_goal_attempts']  += temp_df.home_three_point_field_goal_attempts.tolist()
    stats_dict['three_point_attempt_rate']  += temp_df.home_three_point_attempt_rate.tolist()
    stats_dict['steals']  += temp_df.home_steals.tolist()
    stats_dict['steal_percentage']  += temp_df.home_steal_percentage.tolist()
    stats_dict['ranking']  += temp_df.home_ranking.tolist()
    stats_dict['points']  += temp_df.home_points.tolist()
    stats_dict['personal_fouls']  += temp_df.home_personal_fouls.tolist()
    stats_dict['offensive_rebounds']  += temp_df.home_offensive_rebounds.tolist()
    stats_dict['offensive_rebound_percentage']  += temp_df.home_offensive_rebound_percentage.tolist()
    stats_dict['offensive_rating']  += temp_df.home_offensive_rating.tolist()
    stats_dict['minutes_played']  += temp_df.home_minutes_played.tolist()
    stats_dict['losses']  += temp_df.home_losses.tolist()
    stats_dict['free_throws']  += temp_df.home_free_throws.tolist()
    stats_dict['free_throw_percentage']  += temp_df.home_free_throw_percentage.tolist()
    stats_dict['free_throw_attempts']  += temp_df.home_free_throw_attempts.tolist()
    stats_dict['free_throw_attempt_rate']  += temp_df.home_free_throw_attempt_rate.tolist()
    stats_dict['field_goals']  += temp_df.home_field_goals.tolist()
    stats_dict['field_goal_percentage']  += temp_df.home_field_goal_percentage.tolist()
    stats_dict['field_goal_attempts']  += temp_df.home_field_goal_attempts.tolist()
    stats_dict['effective_field_goal_percentage']  += temp_df.home_effective_field_goal_percentage.tolist()
    stats_dict['defensive_rebounds']  += temp_df.home_defensive_rebounds.tolist()
    stats_dict['defensive_rating']  += temp_df.home_free_throws.tolist()
    stats_dict['blocks']  += temp_df.home_blocks.tolist()
    stats_dict['block_percentage']  += temp_df.home_block_percentage.tolist()
    stats_dict['assists']  += temp_df.home_assists.tolist()
    stats_dict['assist_percentage']  += temp_df.home_assist_percentage.tolist()
    stats_dict['result'] += [0] * len(temp_df)
    stats_dict['opponent_wins']  += temp_df.away_wins.tolist()
    stats_dict['opponent_win_percentage']  += temp_df.away_win_percentage.tolist()
    stats_dict['opponent_two_point_field_goals']  += temp_df.away_two_point_field_goals.tolist()
    stats_dict['opponent_two_point_field_goal_percentage']  += temp_df.away_two_point_field_goal_percentage.tolist() 
    stats_dict['opponent_two_point_field_goal_attempts']  += temp_df.away_two_point_field_goal_attempts.tolist()
    stats_dict['opponent_turnovers']  += temp_df.away_turnovers.tolist()
    stats_dict['opponent_turnover_percentage']  += temp_df.away_turnover_percentage.tolist()
    stats_dict['opponent_true_shooting_percentage']  += temp_df.away_true_shooting_percentage.tolist()
    stats_dict['opponent_total_rebounds']  += temp_df.away_total_rebounds.tolist()
    stats_dict['opponent_total_rebound_percentage']  += temp_df.away_total_rebound_percentage.tolist()
    stats_dict['opponent_three_point_field_goals']  += temp_df.away_three_point_field_goals.tolist()
    stats_dict['opponent_three_point_field_goal_percentage']  += temp_df.away_three_point_field_goal_percentage.tolist()
    stats_dict['opponent_three_point_field_goal_attempts']  += temp_df.away_three_point_field_goal_attempts.tolist()
    stats_dict['opponent_three_point_attempt_rate']  += temp_df.away_three_point_attempt_rate.tolist()
    stats_dict['opponent_steals']  += temp_df.away_steals.tolist()
    stats_dict['opponent_steal_percentage']  += temp_df.away_steal_percentage.tolist()
    stats_dict['opponent_ranking']  += temp_df.away_ranking.tolist()
    stats_dict['opponent_points']  += temp_df.away_points.tolist()
    stats_dict['opponent_personal_fouls']  += temp_df.away_personal_fouls.tolist()
    stats_dict['opponent_offensive_rebounds']  += temp_df.away_offensive_rebounds.tolist()
    stats_dict['opponent_offensive_rebound_percentage']  += temp_df.away_offensive_rebound_percentage.tolist()
    stats_dict['opponent_offensive_rating']  += temp_df.away_offensive_rating.tolist()
    stats_dict['opponent_minutes_played']  += temp_df.away_minutes_played.tolist()
    stats_dict['opponent_losses']  += temp_df.away_losses.tolist()
    stats_dict['opponent_free_throws']  += temp_df.away_free_throws.tolist()
    stats_dict['opponent_free_throw_percentage']  += temp_df.away_free_throw_percentage.tolist()
    stats_dict['opponent_free_throw_attempts']  += temp_df.away_free_throw_attempts.tolist()
    stats_dict['opponent_free_throw_attempt_rate']  += temp_df.away_free_throw_attempt_rate.tolist()
    stats_dict['opponent_field_goals']  += temp_df.away_field_goals.tolist()
    stats_dict['opponent_field_goal_percentage']  += temp_df.away_field_goal_percentage.tolist()
    stats_dict['opponent_field_goal_attempts']  += temp_df.away_field_goal_attempts.tolist()
    stats_dict['opponent_effective_field_goal_percentage']  += temp_df.away_effective_field_goal_percentage.tolist()
    stats_dict['opponent_defensive_rebounds']  += temp_df.away_defensive_rebounds.tolist()
    stats_dict['opponent_defensive_rating']  += temp_df.away_free_throws.tolist()
    stats_dict['opponent_blocks']  += temp_df.away_blocks.tolist()
    stats_dict['opponent_block_percentage']  += temp_df.away_block_percentage.tolist()
    stats_dict['opponent_assists']  += temp_df.away_assists.tolist()
    stats_dict['opponent_assist_percentage']  += temp_df.away_assist_percentage.tolist()
    stats_dict['home'] += [1] * len(temp_df)
    stats_dict['season']  += temp_df.season.tolist()

    #Home/loser Games lost on the road
    temp_df = box_df[(box_df.losing_name == Team) & (box_df.winner == "Home")]
    stats_dict['team']  += temp_df.losing_name.tolist()
    stats_dict['opponent']  += temp_df.winning_name.tolist()
    stats_dict['date']  += temp_df.date.tolist()
    stats_dict['pace']  += temp_df.pace.tolist()
    stats_dict['wins']  += temp_df.away_wins.tolist()
    stats_dict['win_percentage']  += temp_df.away_win_percentage.tolist()
    stats_dict['two_point_field_goals']  += temp_df.away_two_point_field_goals.tolist()
    stats_dict['two_point_field_goal_percentage']  += temp_df.away_two_point_field_goal_percentage.tolist() 
    stats_dict['two_point_field_goal_attempts']  += temp_df.away_two_point_field_goal_attempts.tolist()
    stats_dict['turnovers']  += temp_df.away_turnovers.tolist()
    stats_dict['turnover_percentage']  += temp_df.away_turnover_percentage.tolist()
    stats_dict['true_shooting_percentage']  += temp_df.away_true_shooting_percentage.tolist()
    stats_dict['total_rebounds']  += temp_df.away_total_rebounds.tolist()
    stats_dict['total_rebound_percentage']  += temp_df.away_total_rebound_percentage.tolist()
    stats_dict['three_point_field_goals']  += temp_df.away_three_point_field_goals.tolist()
    stats_dict['three_point_field_goal_percentage']  += temp_df.away_three_point_field_goal_percentage.tolist()
    stats_dict['three_point_field_goal_attempts']  += temp_df.away_three_point_field_goal_attempts.tolist()
    stats_dict['three_point_attempt_rate']  += temp_df.away_three_point_attempt_rate.tolist()
    stats_dict['steals']  += temp_df.away_steals.tolist()
    stats_dict['steal_percentage']  += temp_df.away_steal_percentage.tolist()
    stats_dict['ranking']  += temp_df.away_ranking.tolist()
    stats_dict['points']  += temp_df.away_points.tolist()
    stats_dict['personal_fouls']  += temp_df.away_personal_fouls.tolist()
    stats_dict['offensive_rebounds']  += temp_df.away_offensive_rebounds.tolist()
    stats_dict['offensive_rebound_percentage']  += temp_df.away_offensive_rebound_percentage.tolist()
    stats_dict['offensive_rating']  += temp_df.away_offensive_rating.tolist()
    stats_dict['minutes_played']  += temp_df.away_minutes_played.tolist()
    stats_dict['losses']  += temp_df.away_losses.tolist()
    stats_dict['free_throws']  += temp_df.away_free_throws.tolist()
    stats_dict['free_throw_percentage']  += temp_df.away_free_throw_percentage.tolist()
    stats_dict['free_throw_attempts']  += temp_df.away_free_throw_attempts.tolist()
    stats_dict['free_throw_attempt_rate']  += temp_df.away_free_throw_attempt_rate.tolist()
    stats_dict['field_goals']  += temp_df.away_field_goals.tolist()
    stats_dict['field_goal_percentage']  += temp_df.away_field_goal_percentage.tolist()
    stats_dict['field_goal_attempts']  += temp_df.away_field_goal_attempts.tolist()
    stats_dict['effective_field_goal_percentage']  += temp_df.away_effective_field_goal_percentage.tolist()
    stats_dict['defensive_rebounds']  += temp_df.away_defensive_rebounds.tolist()
    stats_dict['defensive_rating']  += temp_df.away_free_throws.tolist()
    stats_dict['blocks']  += temp_df.away_blocks.tolist()
    stats_dict['block_percentage']  += temp_df.away_block_percentage.tolist()
    stats_dict['assists']  += temp_df.away_assists.tolist()
    stats_dict['assist_percentage']  += temp_df.away_assist_percentage.tolist()
    stats_dict['result'] += [0] * len(temp_df)
    stats_dict['opponent_wins']  += temp_df.home_wins.tolist()
    stats_dict['opponent_win_percentage']  += temp_df.home_win_percentage.tolist()
    stats_dict['opponent_two_point_field_goals']  += temp_df.home_two_point_field_goals.tolist()
    stats_dict['opponent_two_point_field_goal_percentage']  += temp_df.home_two_point_field_goal_percentage.tolist() 
    stats_dict['opponent_two_point_field_goal_attempts']  += temp_df.home_two_point_field_goal_attempts.tolist()
    stats_dict['opponent_turnovers']  += temp_df.home_turnovers.tolist()
    stats_dict['opponent_turnover_percentage']  += temp_df.home_turnover_percentage.tolist()
    stats_dict['opponent_true_shooting_percentage']  += temp_df.home_true_shooting_percentage.tolist()
    stats_dict['opponent_total_rebounds']  += temp_df.home_total_rebounds.tolist()
    stats_dict['opponent_total_rebound_percentage']  += temp_df.home_total_rebound_percentage.tolist()
    stats_dict['opponent_three_point_field_goals']  += temp_df.home_three_point_field_goals.tolist()
    stats_dict['opponent_three_point_field_goal_percentage']  += temp_df.home_three_point_field_goal_percentage.tolist()
    stats_dict['opponent_three_point_field_goal_attempts']  += temp_df.home_three_point_field_goal_attempts.tolist()
    stats_dict['opponent_three_point_attempt_rate']  += temp_df.home_three_point_attempt_rate.tolist()
    stats_dict['opponent_steals']  += temp_df.home_steals.tolist()
    stats_dict['opponent_steal_percentage']  += temp_df.home_steal_percentage.tolist()
    stats_dict['opponent_ranking']  += temp_df.home_ranking.tolist()
    stats_dict['opponent_points']  += temp_df.home_points.tolist()
    stats_dict['opponent_personal_fouls']  += temp_df.home_personal_fouls.tolist()
    stats_dict['opponent_offensive_rebounds']  += temp_df.home_offensive_rebounds.tolist()
    stats_dict['opponent_offensive_rebound_percentage']  += temp_df.home_offensive_rebound_percentage.tolist()
    stats_dict['opponent_offensive_rating']  += temp_df.home_offensive_rating.tolist()
    stats_dict['opponent_minutes_played']  += temp_df.home_minutes_played.tolist()
    stats_dict['opponent_losses']  += temp_df.home_losses.tolist()
    stats_dict['opponent_free_throws']  += temp_df.home_free_throws.tolist()
    stats_dict['opponent_free_throw_percentage']  += temp_df.home_free_throw_percentage.tolist()
    stats_dict['opponent_free_throw_attempts']  += temp_df.home_free_throw_attempts.tolist()
    stats_dict['opponent_free_throw_attempt_rate']  += temp_df.home_free_throw_attempt_rate.tolist()
    stats_dict['opponent_field_goals']  += temp_df.home_field_goals.tolist()
    stats_dict['opponent_field_goal_percentage']  += temp_df.home_field_goal_percentage.tolist()
    stats_dict['opponent_field_goal_attempts']  += temp_df.home_field_goal_attempts.tolist()
    stats_dict['opponent_effective_field_goal_percentage']  += temp_df.home_effective_field_goal_percentage.tolist()
    stats_dict['opponent_defensive_rebounds']  += temp_df.home_defensive_rebounds.tolist()
    stats_dict['opponent_defensive_rating']  += temp_df.home_free_throws.tolist()
    stats_dict['opponent_blocks']  += temp_df.home_blocks.tolist()
    stats_dict['opponent_block_percentage']  += temp_df.home_block_percentage.tolist()
    stats_dict['opponent_assists']  += temp_df.home_assists.tolist()
    stats_dict['opponent_assist_percentage']  += temp_df.home_assist_percentage.tolist()
    stats_dict['home'] += [0] * len(temp_df)
    stats_dict['season']  += temp_df.season.tolist()

100%|██████████| 990/990 [00:59<00:00, 16.68it/s]


In [188]:
final_df = pd.DataFrame.from_dict(stats_dict)
final_df['date'] = pd.to_datetime(final_df['date'])
final_df.drop_duplicates(inplace = True)
final_df.sort_values('date',inplace = True)
final_df[final_df.team == 'Purdue']

,team,home,opponent,date,season,pace,result,wins,win_percentage,two_point_field_goals,two_point_field_goal_percentage,two_point_field_goal_attempts,turnovers,turnover_percentage,true_shooting_percentage,total_rebounds,total_rebound_percentage,three_point_field_goals,three_point_field_goal_percentage,three_point_field_goal_attempts,three_point_attempt_rate,steals,steal_percentage,ranking,points,personal_fouls,offensive_rebounds,offensive_rebound_percentage,offensive_rating,minutes_played,losses,free_throws,free_throw_percentage,free_throw_attempts,free_throw_attempt_rate,field_goals,field_goal_percentage,field_goal_attempts,effective_field_goal_percentage,defensive_rebounds,defensive_rating,blocks,block_percentage,assists,assist_percentage,opponent_wins,opponent_win_percentage,opponent_two_point_field_goals,opponent_two_point_field_goal_percentage,opponent_two_point_field_goal_attempts,opponent_turnovers,opponent_turnover_percentage,opponent_true_shooting_percentage,opponent_total_rebounds,opponent_total_rebound_percentage,opponent_three_point_field_goals,opponent_three_point_field_goal_percentage,opponent_three_point_field_goal_attempts,opponent_three_point_attempt_rate,opponent_steals,opponent_steal_percentage,opponent_ranking,opponent_points,opponent_personal_fouls,opponent_offensive_rebounds,opponent_offensive_rebound_percentage,opponent_offensive_rating,opponent_minutes_played,opponent_losses,opponent_free_throws,opponent_free_throw_percentage,opponent_free_throw_attempts,opponent_free_throw_attempt_rate,opponent_field_goals,opponent_field_goal_percentage,opponent_field_goal_attempts,opponent_effective_field_goal_percentage,opponent_defensive_rebounds,opponent_defensive_rating,opponent_blocks,opponent_block_percentage,opponent_assists,opponent_assist_percentage
111592,Purdue,1,Northern Kentucky,2013-11-08,2014,66.4,1,1,1.0,20.0,0.465,43.0,12,15.2,0.570,32,50.8,7.0,0.467,15.0,0.259,6,9.1,0.0,77,15,15,46.9,116.7,200.0,0,16,0.800,20,0.345,27,0.466,58,0.526,17,16,4,10.8,12,44.4,0,0.000,14.0,0.378,37.0,14,17.1,0.557,31,49.2,13.0,0.500,26.0,0.413,7,10.6,0.0,76,17,14,45.2,115.2,200.0,1,9,0.818,11,0.175,27,0.429,63,0.532,17,9,5,11.6,18,66.7
111585,Purdue,1,Central Connecticut,2013-11-13,2014,83.2,1,2,1.0,34.0,0.618,55.0,9,9.1,0.603,48,57.8,9.0,0.375,24.0,0.304,6,7.2,0.0,109,18,19,47.5,131.3,200.0,0,14,0.583,24,0.304,43,0.544,79,0.601,29,14,13,22.8,22,51.2,0,0.000,23.0,0.404,57.0,11,11.1,0.410,35,42.2,4.0,0.190,21.0,0.269,7,8.4,0.0,73,19,14,32.6,88.0,200.0,2,15,0.652,23,0.295,27,0.346,78,0.372,21,15,3,5.5,15,55.6
111596,Purdue,1,Rider,2013-11-17,2014,72.2,1,3,1.0,28.0,0.609,46.0,9,10.8,0.535,30,49.2,3.0,0.200,15.0,0.246,9,12.5,0.0,81,27,13,36.1,112.5,200.0,0,16,0.516,31,0.508,31,0.508,61,0.533,17,16,5,13.5,15,48.4,0,0.000,16.0,0.432,37.0,13,16.3,0.568,31,50.8,7.0,0.438,16.0,0.302,2,2.8,0.0,77,27,8,32.0,106.9,200.0,2,24,0.774,31,0.585,23,0.434,53,0.500,23,24,6,13.0,11,47.8
111586,Purdue,1,Eastern Illinois,2013-11-20,2014,77.3,1,4,1.0,24.0,0.585,41.0,10,11.6,0.538,40,54.1,4.0,0.200,20.0,0.328,12,15.6,0.0,83,20,11,31.4,107.8,200.0,0,23,0.676,34,0.557,28,0.459,61,0.492,29,23,7,17.1,16,57.1,2,0.500,16.0,0.390,41.0,19,21.7,0.396,34,45.9,3.0,0.176,17.0,0.293,5,6.5,0.0,55,22,10,25.6,71.4,200.0,2,14,0.583,24,0.414,19,0.328,58,0.353,24,14,8,19.5,11,57.9
111598,Purdue,1,Siena,2013-11-24,2014,83.2,1,5,1.0,22.0,0.468,47.0,15,15.9,0.503,42,51.2,5.0,0.313,16.0,0.254,6,7.2,0.0,81,19,13,32.5,97.6,200.0,0,22,0.595,37,0.587,27,0.429,63,0.468,29,22,12,19.7,11,40.7,2,0.333,27.0,0.443,61.0,14,14.6,0.440,40,48.8,1.0,0.100,10.0,0.141,10,12.0,0.0,73,25,13,31.0,88.0,200.0,4,16,0.640,25,0.352,28,0.394,71,0.401,27,16,7,14.9,15,53.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [190]:
final_df.isna().sum()

team                          0
home                          0
opponent                      0
date                          0
season                        0
                             ..
opponent_defensive_rating     0
opponent_blocks               0
opponent_block_percentage     0
opponent_assists              0
opponent_assist_percentage    0
Length: 83, dtype: int64

In [191]:
final_df.to_csv('mldata.csv')

In [192]:
agg_cols = ['abbreviation', 'assist_percentage', 'assists', 'away_losses',
       'away_wins', 'block_percentage', 'blocks', 'conference',
       'conference_losses', 'conference_wins', 'defensive_rebounds',
       'effective_field_goal_percentage', 'field_goal_attempts',
       'field_goal_percentage', 'field_goals', 'free_throw_attempt_rate',
       'free_throw_attempts', 'free_throw_percentage', 'free_throws',
       'free_throws_per_field_goal_attempt', 'games_played', 'home_losses',
       'home_wins', 'losses', 'minutes_played', 'name', 'net_rating',
       'offensive_rating', 'offensive_rebound_percentage',
       'offensive_rebounds', 'opp_assist_percentage', 'opp_assists',
       'opp_block_percentage', 'opp_blocks', 'opp_defensive_rebounds',
       'opp_effective_field_goal_percentage', 'opp_field_goal_attempts',
       'opp_field_goal_percentage', 'opp_field_goals',
       'opp_free_throw_attempt_rate', 'opp_free_throw_attempts',
       'opp_free_throw_percentage', 'opp_free_throws',
       'opp_free_throws_per_field_goal_attempt', 'opp_offensive_rating',
       'opp_offensive_rebound_percentage', 'opp_offensive_rebounds',
       'opp_personal_fouls', 'opp_points', 'opp_steal_percentage',
       'opp_steals', 'opp_three_point_attempt_rate',
       'opp_three_point_field_goal_attempts',
       'opp_three_point_field_goal_percentage', 'opp_three_point_field_goals',
       'opp_two_point_field_goal_attempts',
       'opp_two_point_field_goal_percentage', 'opp_two_point_field_goals',
       'opp_total_rebound_percentage', 'opp_total_rebounds',
       'opp_true_shooting_percentage', 'opp_turnover_percentage',
       'opp_turnovers', 'pace', 'personal_fouls', 'points',
       'simple_rating_system', 'steal_percentage', 'steals',
       'strength_of_schedule', 'three_point_attempt_rate',
       'three_point_field_goal_attempts', 'three_point_field_goal_percentage',
       'three_point_field_goals', 'two_point_field_goal_attempts',
       'two_point_field_goal_percentage', 'two_point_field_goals',
       'total_rebound_percentage', 'total_rebounds',
       'true_shooting_percentage', 'turnover_percentage', 'turnovers',
       'win_percentage', 'wins']

In [193]:
df_agg = pd.DataFrame(columns=agg_cols)
seasons = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]
for season in seasons:
    print("Getting Season Data...")
    for t in Teams(season):
        try:
            temp_df = t.dataframe
            temp_df['season'] = season
            df_agg = pd.concat([df_agg,temp_df],axis=0)
        except:
            pass
        
df_agg

Getting Season Data...
Getting Season Data...
Getting Season Data...
Getting Season Data...
Getting Season Data...
Getting Season Data...
Getting Season Data...
Getting Season Data...
Getting Season Data...
Getting Season Data...
Getting Season Data...


,abbreviation,assist_percentage,assists,away_losses,away_wins,block_percentage,blocks,conference,conference_losses,conference_wins,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,free_throws_per_field_goal_attempt,games_played,home_losses,home_wins,losses,minutes_played,name,net_rating,offensive_rating,offensive_rebound_percentage,offensive_rebounds,opp_assist_percentage,opp_assists,opp_block_percentage,opp_blocks,opp_defensive_rebounds,opp_effective_field_goal_percentage,opp_field_goal_attempts,opp_field_goal_percentage,opp_field_goals,opp_free_throw_attempt_rate,opp_free_throw_attempts,opp_free_throw_percentage,opp_free_throws,opp_free_throws_per_field_goal_attempt,opp_offensive_rating,opp_offensive_rebound_percentage,opp_offensive_rebounds,opp_personal_fouls,opp_points,opp_steal_percentage,opp_steals,opp_three_point_attempt_rate,opp_three_point_field_goal_attempts,opp_three_point_field_goal_percentage,opp_three_point_field_goals,opp_two_point_field_goal_attempts,opp_two_point_field_goal_percentage,opp_two_point_field_goals,opp_total_rebound_percentage,opp_total_rebounds,opp_true_shooting_percentage,opp_turnover_percentage,opp_turnovers,pace,personal_fouls,points,simple_rating_system,steal_percentage,steals,strength_of_schedule,three_point_attempt_rate,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_percentage,wins,season
AIR-FORCE,AIR-FORCE,68.4,494,11,3,8.7,83,mwc,10,6,731,0.540,1532,0.471,722,0.387,593,0.705,418,0.273,32,5,12,16,1295,Air Force,None,102.5,20.4,187,63.7,449,10.3,100,729,0.482,1737,0.406,705,0.326,567,0.716,406,0.234,None,34.3,381,587,2082,8.6,174,0.454,788,0.338,266,949,0.463,439,54.7,1110,0.519,16.9,407,62.5,563,2074,2.71,9.9,200,3.31,0.367,563,0.377,212,969,0.526,510,45.3,918,0.572,17.5,386,0.500,16,2011.0
AKRON,AKRON,58.3,520,9,5,9.2,129,mac,7,9,875,0.499,2073,0.430,892,0.313,649,0.704,457,0.220,36,2,14,13,1465,Akron,None,102.9,28.3,363,51.1,425,6.6,85,920,0.464,2004,0.415,832,0.383,768,0.660,507,0.253,None,32.7,425,655,2365,8.8,216,0.302,606,0.320,194,1398,0.456,638,52.1,1345,0.499,17.8,514,67.0,692,2524,2.15,9.8,240,-1.02,0.379,785,0.361,283,1288,0.473,609,47.9,1238,0.530,15.4,432,0.639,23,2011.0
ALABAMA-AM,ALABAMA-AM,53.6,339,10,4,13.1,151,swac,8,10,711,0.435,1598,0.396,633,0.418,668,0.624,417,0.261,28,4,9,15,1125,Alabama A&M,None,89.8,31.7,347,55.2,363,10.1,117,749,0.457,1594,0.413,658,0.415,662,0.624,413,0.259,None,33.6,360,565,1870,11.0,221,0.277,441,0.320,141,1153,0.448,517,51.2,1109,0.490,19.6,465,71.6,562,1808,-15.19,11.6,233,-11.75,0.272,434,0.288,125,1164,0.436,508,48.8,1058,0.472,18.8,444,0.464,13,2011.0
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,55.6,420,5,8,8.2,97,cusa,4,12,757,0.508,1725,0.438,756,0.297,513,0.717,368,0.213,31,2,13,9,1270,Alabama-Birmingham,None,105.7,33.4,352,48.5,339,9.9,102,702,0.459,1692,0.413,699,0.324,549,0.703,386,0.228,None,30.6,333,539,1940,9.3,186,0.297,503,0.310,156,1189,0.457,543,48.3,1035,0.497,16.5,385,63.1,514,2119,8.55,8.2,164,2.78,0.405,698,0.342,239,1027,0.503,517,51.7,1109,0.538,16.4,387,0.710,22,2011.0
ALABAMA-STATE,ALABAMA-STATE,55.5,404,14,4,13.1,163,swac,7,11,811,0.440,1841,0.395,728,0.473,870,0.603,525,0.285,35,3,10,18,1410,Alabama State,None,91.5,35.7,460,55.9,400,6.8,86,829,0.454,1773,0.403,715,0.515,913,0.683,624,0.352,None,33.4,406,722,2234,11.2,262,0.296,525,0.343,180,1248,0.429,535,49.3,1235,0.506,19.8,545,66.5,777,2146,-13.37,10.9,255,-10.31,0.317,583,0.283,165,1258,0.448,563,50.7,1271,0.476,19.6,551,0.486,17,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [396]:
in_agg = ['Southern Methodist','Brigham Young','University of California','Illinois-Chicago','Connecticut','Massachusetts','Nevada-Las Vegas','North Carolina',
         'North Carolina-Wilmington','North Carolina State','Pittsburgh','Louisiana State','Texas Christian','Southern California','Virginia Commonwealth']
to_change = ['SMU','BYU','California','UIC','UConn','UMass','UNLV','UNC','UNC Wilmington','UNC Asheville','NC State','Pitt','LSU','TCU','USC','VCU']
rename_dict = dict(zip(in_agg,to_change))
df_agg['name'] = df_agg.name.replace(rename_dict)
df_agg

,abbreviation,assist_percentage,assists,away_losses,away_wins,block_percentage,blocks,conference,conference_losses,conference_wins,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,free_throws_per_field_goal_attempt,games_played,home_losses,home_wins,losses,minutes_played,name,net_rating,offensive_rating,offensive_rebound_percentage,offensive_rebounds,opp_assist_percentage,opp_assists,opp_block_percentage,opp_blocks,opp_defensive_rebounds,opp_effective_field_goal_percentage,opp_field_goal_attempts,opp_field_goal_percentage,opp_field_goals,opp_free_throw_attempt_rate,opp_free_throw_attempts,opp_free_throw_percentage,opp_free_throws,opp_free_throws_per_field_goal_attempt,opp_offensive_rating,opp_offensive_rebound_percentage,opp_offensive_rebounds,opp_personal_fouls,opp_points,opp_steal_percentage,opp_steals,opp_three_point_attempt_rate,opp_three_point_field_goal_attempts,opp_three_point_field_goal_percentage,opp_three_point_field_goals,opp_two_point_field_goal_attempts,opp_two_point_field_goal_percentage,opp_two_point_field_goals,opp_total_rebound_percentage,opp_total_rebounds,opp_true_shooting_percentage,opp_turnover_percentage,opp_turnovers,pace,personal_fouls,points,simple_rating_system,steal_percentage,steals,strength_of_schedule,three_point_attempt_rate,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_percentage,wins,season
AIR-FORCE,AIR-FORCE,68.4,494,11,3,8.7,83,mwc,10,6,731,0.540,1532,0.471,722,0.387,593,0.705,418,0.273,32,5,12,16,1295,Air Force,None,102.5,20.4,187,63.7,449,10.3,100,729,0.482,1737,0.406,705,0.326,567,0.716,406,0.234,None,34.3,381,587,2082,8.6,174,0.454,788,0.338,266,949,0.463,439,54.7,1110,0.519,16.9,407,62.5,563,2074,2.71,9.9,200,3.31,0.367,563,0.377,212,969,0.526,510,45.3,918,0.572,17.5,386,0.500,16,2011.0
AKRON,AKRON,58.3,520,9,5,9.2,129,mac,7,9,875,0.499,2073,0.430,892,0.313,649,0.704,457,0.220,36,2,14,13,1465,Akron,None,102.9,28.3,363,51.1,425,6.6,85,920,0.464,2004,0.415,832,0.383,768,0.660,507,0.253,None,32.7,425,655,2365,8.8,216,0.302,606,0.320,194,1398,0.456,638,52.1,1345,0.499,17.8,514,67.0,692,2524,2.15,9.8,240,-1.02,0.379,785,0.361,283,1288,0.473,609,47.9,1238,0.530,15.4,432,0.639,23,2011.0
ALABAMA-AM,ALABAMA-AM,53.6,339,10,4,13.1,151,swac,8,10,711,0.435,1598,0.396,633,0.418,668,0.624,417,0.261,28,4,9,15,1125,Alabama A&M,None,89.8,31.7,347,55.2,363,10.1,117,749,0.457,1594,0.413,658,0.415,662,0.624,413,0.259,None,33.6,360,565,1870,11.0,221,0.277,441,0.320,141,1153,0.448,517,51.2,1109,0.490,19.6,465,71.6,562,1808,-15.19,11.6,233,-11.75,0.272,434,0.288,125,1164,0.436,508,48.8,1058,0.472,18.8,444,0.464,13,2011.0
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,55.6,420,5,8,8.2,97,cusa,4,12,757,0.508,1725,0.438,756,0.297,513,0.717,368,0.213,31,2,13,9,1270,Alabama-Birmingham,None,105.7,33.4,352,48.5,339,9.9,102,702,0.459,1692,0.413,699,0.324,549,0.703,386,0.228,None,30.6,333,539,1940,9.3,186,0.297,503,0.310,156,1189,0.457,543,48.3,1035,0.497,16.5,385,63.1,514,2119,8.55,8.2,164,2.78,0.405,698,0.342,239,1027,0.503,517,51.7,1109,0.538,16.4,387,0.710,22,2011.0
ALABAMA-STATE,ALABAMA-STATE,55.5,404,14,4,13.1,163,swac,7,11,811,0.440,1841,0.395,728,0.473,870,0.603,525,0.285,35,3,10,18,1410,Alabama State,None,91.5,35.7,460,55.9,400,6.8,86,829,0.454,1773,0.403,715,0.515,913,0.683,624,0.352,None,33.4,406,722,2234,11.2,262,0.296,525,0.343,180,1248,0.429,535,49.3,1235,0.506,19.8,545,66.5,777,2146,-13.37,10.9,255,-10.31,0.317,583,0.283,165,1258,0.448,563,50.7,1271,0.476,19.6,551,0.486,17,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [397]:
simple_df = box_df[['date','winning_name','losing_name','season','winner','home_points','away_points']].drop_duplicates().sort_values('date')
simple_df

,date,winning_name,losing_name,season,winner,home_points,away_points
2014-04-01-florida-state,"April 1, 2014",Minnesota,Florida State,2014,Away,64,67
2014-04-01-clemson,"April 1, 2014",SMU,Clemson,2014,Away,59,65
2014-04-01-murray-state,"April 1, 2014",Murray State,Pacific,2014,Home,98,75
2014-04-01-virginia-military-institute,"April 1, 2014",Yale,VMI,2014,Away,62,75
2015-04-01-louisiana-monroe,"April 1, 2015",Loyola (IL),Louisiana-Monroe,2015,Away,62,63
...,...,...,...,...,...,...,...
2019-11-09-19-wofford,"November 9, 2019",Wofford,High Point,2020,Home,89,61
2019-11-09-14-temple,"November 9, 2019",Temple,Morgan State,2020,Home,75,57
2019-11-09-19-saint-louis,"November 9, 2019",Saint Louis,Valparaiso,2020,Home,81,70
2019-11-09-14-st-francis-ny,"November 9, 2019",St. Francis (NY),Lafayette,2020,Home,73,72


In [196]:
simple_dict = {key:[] for key in ('id','date','season','team','opponent','home','result','points_for','points_against')}
for i in tqdm(simple_df.index):
    temp_df = simple_df.loc[i]
    j = [int(temp_df['home_points']),int(temp_df['away_points'])]
    simple_dict['id'].append(i)
    simple_dict['id'].append(i)
    simple_dict['date'].append(temp_df['date'])
    simple_dict['season'].append(temp_df['season'])
    simple_dict['team'].append(temp_df['winning_name'])
    simple_dict['opponent'].append(temp_df['losing_name'])
    simple_dict['home'].append(1 if temp_df['winner'] == 'Home' else 0)
    simple_dict['result'].append(1)
    simple_dict['points_for'].append(max(j))
    simple_dict['points_against'].append(min(j))
    simple_dict['date'].append(temp_df['date'])
    simple_dict['season'].append(temp_df['season'])
    simple_dict['team'].append(temp_df['losing_name'])
    simple_dict['opponent'].append(temp_df['winning_name'])
    simple_dict['home'].append(1 if temp_df['winner'] == 'Away' else 0)
    simple_dict['result'].append(0)
    simple_dict['points_for'].append(min(j))
    simple_dict['points_against'].append(max(j))

100%|██████████| 44219/44219 [00:06<00:00, 6398.91it/s]


In [197]:
ml_df_l1 = pd.DataFrame.from_dict(simple_dict)
ml_df_l1

,id,date,season,team,opponent,home,result,points_for,points_against
0,2014-04-01-florida-state,"April 1, 2014",2014,Minnesota,Florida State,0,1,67,64
1,2014-04-01-florida-state,"April 1, 2014",2014,Florida State,Minnesota,1,0,64,67
2,2014-04-01-clemson,"April 1, 2014",2014,SMU,Clemson,0,1,65,59
3,2014-04-01-clemson,"April 1, 2014",2014,Clemson,SMU,1,0,59,65
4,2014-04-01-murray-state,"April 1, 2014",2014,Murray State,Pacific,1,1,98,75
...,...,...,...,...,...,...,...,...,...
88433,2019-11-09-19-saint-louis,"November 9, 2019",2020,Valparaiso,Saint Louis,0,0,70,81
88434,2019-11-09-14-st-francis-ny,"November 9, 2019",2020,St. Francis (NY),Lafayette,1,1,73,72
88435,2019-11-09-14-st-francis-ny,"November 9, 2019",2020,Lafayette,St. Francis (NY),0,0,72,73
88436,2019-11-09-16-western-kentucky,"November 9, 2019",2020,Western Kentucky,Austin Peay,1,1,97,75


In [398]:
all_cols = ['assist_percentage',
'block_percentage',
'effective_field_goal_percentage',
'field_goal_percentage',
'free_throw_attempt_rate',
'free_throw_percentage',
'name',
'net_rating',
'offensive_rating',
'offensive_rebound_percentage',
'allowed_assist_percentage',
'allowed_block_percentage',
'allowed_effective_field_goal_percentage',
'allowed_field_goal_percentage',
'allowed_free_throw_attempt_rate',
'allowed_free_throw_percentage',
'allowed_offensive_rating',
'allowed_offensive_rebound_percentage',
'allowed_steal_percentage',
'allowed_three_point_attempt_rate',
'allowed_three_point_field_goal_percentage',
'allowed_two_point_field_goal_percentage',
'allowed_total_rebound_percentage',
'allowed_true_shooting_percentage',
'allowed_turnover_percentage',
'pace',
'simple_rating_system',
'steal_percentage',
'three_point_attempt_rate',
'three_point_field_goal_percentage',
'two_point_field_goal_percentage',
'total_rebound_percentage',
'true_shooting_percentage',
'turnover_percentage',
'win_percentage',
'opp_assist_percentage',
'opp_block_percentage',
'opp_effective_field_goal_percentage',
'opp_field_goal_percentage',
'opp_free_throw_attempt_rate',
'opp_free_throw_percentage',
'opp_name',
'opp_net_rating',
'opp_offensive_rating',
'opp_offensive_rebound_percentage',
'opp_pace',
'opp_simple_rating_system',
'opp_steal_percentage',
'opp_three_point_attempt_rate',
'opp_three_point_field_goal_percentage',
'opp_two_point_field_goal_percentage',
'opp_total_rebound_percentage',
'opp_true_shooting_percentage',
'opp_turnover_percentage',
'opp_win_percentage',
'opp_assist_percentage_allowed',
'opp_block_percentage_allowed',
'opp_effective_field_goal_percentage_allowed',
'opp_field_goal_percentage_allowed',
'opp_free_throw_attempt_rate_allowed',
'opp_free_throw_percentage_allowed',
'opp_offensive_rating_allowed',
'opp_offensive_rebound_percentage_allowed',
'opp_steal_percentage_allowed',
'opp_three_point_attempt_rate_allowed',
'opp_three_point_field_goal_percentage_allowed',
'opp_two_point_field_goal_percentage_allowed',
'opp_total_rebound_percentage_allowed',
'opp_true_shooting_percentage_allowed',
'opp_turnover_percentage_allowed',
'id',
'date',
'season',
'team',
'opponent',
'home',
'result',
'points_for',
'points_against']
master_dict = {key:[] for key in all_cols}

for i in tqdm(ml_df_l1.id.unique()):
    temp_df = ml_df_l1[ml_df_l1.id == i]
    season = temp_df.season.unique()[0]
    team_1 = temp_df.team.tolist()[0]
    team_2 = temp_df.team.tolist()[1]
    temp_df_2 = df_agg.loc[(df_agg.season == season) & (df_agg.name == team_1)].loc[:,df_agg.columns[df_agg.columns.str.contains('percen') | df_agg.columns.str.contains('rat') | (df_agg.columns == 'pace') | (df_agg.columns == 'name')]]
    temp_df_3 = df_agg.loc[(df_agg.season == season) & (df_agg.name == team_2)].loc[:,df_agg.columns[df_agg.columns.str.contains('percen') | df_agg.columns.str.contains('rat') | (df_agg.columns == 'pace') | (df_agg.columns == 'name')]]
    if (len(temp_df_2)>0) & (len(temp_df_3)>0):
        old_names = temp_df_2.columns[temp_df_2.columns.str.contains('opp_')].to_list()
        new_names = [i.replace('opp','allowed') for i in temp_df_2.columns[temp_df_2.columns.str.contains('opp_')].to_list()]
        name_dict = dict(zip(old_names,new_names))
        temp_df_2_dict = temp_df_2.rename(columns = name_dict).to_dict(orient = 'list')
        temp_df_2_dict.update(temp_df_3.iloc[0,:10].rename(dict(zip(temp_df_3.iloc[0,:10].index,['opp_'+str(i) for i in temp_df_3.iloc[0,:10].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_2_dict.update(temp_df_3.iloc[0,25:].rename(dict(zip(temp_df_3.iloc[0,25:].index,['opp_'+str(i) for i in temp_df_3.iloc[0,25:].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_2_dict.update(temp_df_3.iloc[0,10:25].rename(dict(zip(temp_df_3.iloc[0,10:25].index,[i + "_allowed" for i in temp_df_3.iloc[0,10:25].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_2_dict
        temp_df_3_dict = temp_df_3.rename(columns = name_dict).to_dict(orient = 'list')
        temp_df_3_dict.update(temp_df_2.iloc[0,:10].rename(dict(zip(temp_df_2.iloc[0,:10].index,['opp_'+str(i) for i in temp_df_2.iloc[0,:10].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_3_dict.update(temp_df_2.iloc[0,25:].rename(dict(zip(temp_df_2.iloc[0,25:].index,['opp_'+str(i) for i in temp_df_2.iloc[0,25:].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_3_dict.update(temp_df_2.iloc[0,10:25].rename(dict(zip(temp_df_2.iloc[0,10:25].index,[i + "_allowed" for i in temp_df_2.iloc[0,10:25].index]))).to_frame().transpose().to_dict(orient = 'list'))
        for i in temp_df_3_dict.keys():
            temp_df_2_dict[i]+=(temp_df_3_dict[i])
        temp_df_2_dict.update(temp_df.to_dict(orient = 'list'))
        for i in temp_df_2_dict.keys():
            master_dict[i] += temp_df_2_dict[i]

ml_df_l3 = pd.DataFrame.from_dict(master_dict)
ml_df_l3

100%|██████████| 44219/44219 [18:16<00:00, 40.32it/s]


,assist_percentage,block_percentage,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,name,net_rating,offensive_rating,offensive_rebound_percentage,allowed_assist_percentage,allowed_block_percentage,allowed_effective_field_goal_percentage,allowed_field_goal_percentage,allowed_free_throw_attempt_rate,allowed_free_throw_percentage,allowed_offensive_rating,allowed_offensive_rebound_percentage,allowed_steal_percentage,allowed_three_point_attempt_rate,allowed_three_point_field_goal_percentage,allowed_two_point_field_goal_percentage,allowed_total_rebound_percentage,allowed_true_shooting_percentage,allowed_turnover_percentage,pace,simple_rating_system,steal_percentage,three_point_attempt_rate,three_point_field_goal_percentage,two_point_field_goal_percentage,total_rebound_percentage,true_shooting_percentage,turnover_percentage,win_percentage,opp_assist_percentage,opp_block_percentage,opp_effective_field_goal_percentage,opp_field_goal_percentage,opp_free_throw_attempt_rate,opp_free_throw_percentage,opp_name,opp_net_rating,opp_offensive_rating,opp_offensive_rebound_percentage,opp_pace,opp_simple_rating_system,opp_steal_percentage,opp_three_point_attempt_rate,opp_three_point_field_goal_percentage,opp_two_point_field_goal_percentage,opp_total_rebound_percentage,opp_true_shooting_percentage,opp_turnover_percentage,opp_win_percentage,opp_assist_percentage_allowed,opp_block_percentage_allowed,opp_effective_field_goal_percentage_allowed,opp_field_goal_percentage_allowed,opp_free_throw_attempt_rate_allowed,opp_free_throw_percentage_allowed,opp_offensive_rating_allowed,opp_offensive_rebound_percentage_allowed,opp_steal_percentage_allowed,opp_three_point_attempt_rate_allowed,opp_three_point_field_goal_percentage_allowed,opp_two_point_field_goal_percentage_allowed,opp_total_rebound_percentage_allowed,opp_true_shooting_percentage_allowed,opp_turnover_percentage_allowed,id,date,season,team,opponent,home,result,points_for,points_against
0,57.5,11.8,0.513,0.447,0.398,0.742,Minnesota,None,108.9,31.4,55.8,12.4,0.486,0.426,0.398,0.732,None,31.1,9.1,0.354,0.343,0.471,49.5,0.532,16.3,64.5,11.75,11.6,0.377,0.352,0.505,50.5,0.556,15.4,0.658,44.1,13.9,0.517,0.466,0.456,0.722,Florida State,None,106.6,34.8,66.1,12.63,8.9,0.268,0.380,0.497,50.8,0.560,18.3,0.611,48.8,9.4,0.454,0.399,0.362,0.691,None,35.8,10.0,0.336,0.329,0.435,49.2,0.494,15.8,2014-04-01-florida-state,"April 1, 2014",2014,Minnesota,Florida State,0,1,67,64
1,44.1,13.9,0.517,0.466,0.456,0.722,Florida State,None,106.6,34.8,48.8,9.4,0.454,0.399,0.362,0.691,None,35.8,10.0,0.336,0.329,0.435,49.2,0.494,15.8,66.1,12.63,8.9,0.268,0.380,0.497,50.8,0.560,18.3,0.611,57.5,11.8,0.513,0.447,0.398,0.742,Minnesota,None,108.9,31.4,64.5,11.75,11.6,0.377,0.352,0.505,50.5,0.556,15.4,0.658,55.8,12.4,0.486,0.426,0.398,0.732,None,31.1,9.1,0.354,0.343,0.471,49.5,0.532,16.3,2014-04-01-florida-state,"April 1, 2014",2014,Florida State,Minnesota,1,0,64,67
2,59.8,13.7,0.528,0.483,0.438,0.683,SMU,None,107.6,35.0,57.0,9.4,0.448,0.385,0.387,0.694,None,30.5,11.8,0.391,0.321,0.427,46.5,0.492,18.1,66.0,13.73,11.6,0.237,0.379,0.516,53.5,0.561,17.4,0.730,48.1,16.2,0.474,0.424,0.361,0.744,Clemson,None,103.9,33.8,59.9,11.08,7.6,0.326,0.310,0.479,51.4,0.519,15.4,0.639,47.9,8.2,0.439,0.395,0.286,0.665,None,32.5,8.9,0.306,0.285,0.443,48.6,0.470,14.5,2014-04-01-clemson,"April 1, 2014",2014,SMU,Clemson,0,1,65,59
3,48.1,16.2,0.474,0.424,0.361,0.744,Clemson,None,103.9,33.8,47.9,8.2,0.439,0.395,0.286,0.665,None,32.5,8.9,0.306,0.285,0.443,48.6,0.470,14.5,59.9,11.08,7.6,0.326,0.310,0.479,51.4,0.519,15.4,0.639,59.8,13.7,0.528,0.483,0.438,0.683,SMU,None,107.6,35.0,66.0,13.73,11.6,0.237,0.379,0.516,53.5,0.561,17.4,0.730,57.0,9.4,0.448,0.385,0.387,0.694,None,30.5,11.8,0.391,0.321,0.427,46.5,0.492,18.1,2014-04-01-clemson,"April 1, 2014",2014,Clemson,SMU,1,0,59,65
4,49.5,10.3,0.541,0.475,0.465,0.672,Murray State,None,113.0,34.3,46.2,8.7,0.491,0.445,0.347,0.671,None,32.4,8.8,0.285,0.322,0.494,48.5,0.522

In [399]:
ml_df_l3.to_csv('ml_df_l3.csv')

In [431]:
ml_df_l3['team'] = ml_df_l3['team'].map(lambda x: x.replace("State",'St.'))

In [489]:
browser = login("xxx", 'xxx')
kp_df = kp.get_fourfactors(browser,2022)
kp_df = kp_df.loc[:,kp_df.columns[~kp_df.columns.str.contains('rank',case = False)]]

missing = []
for i in ml_df_l3.team.str.replace("State",'St.').unique().tolist():
    if i not in kp_df.Team.unique().tolist():
        missing.append(i)
in_kp = ['Loyola Chicago','Louisiana Monroe','North Carolina','Miami FL','Connecticut','Charleston','UAB','Albany','St. Francis NY',
        'Prairie View A&M','Savannah St.','Nebraska Omaha','Maryland Eastern Shore','UT Arlington','Loyola MD','N.C. State','Pittsburgh','The Citadel',
        'Grambling St.',"St. John's",'UC Irvine','Cal Baptist','Bowling Green','Bethune Cookman','Gardner Webb','FIU','UT Rio Grande Valley','St. Francis PA',
        'UC Riverside','Purdue Fort Wayne','Massachusetts','UC Davis','Miami OH','Texas A&M Corpus Chris','Arkansas Pine Bluff','UC San Diego']

kp_df['Team'] = kp_df.Team.replace(dict(zip(in_kp,missing)))
kp_df

C:\Users\Frank\anaconda3\lib\site-packages\kenpompy\summary.py:111: FutureWarning: The default value of regex will change from True to False in a future version.
  ff_df['Team'] = ff_df['Team'].str.replace('\d+', '')


,Team,Conference,AdjTempo,AdjOE,Off-eFG%,Off-TO%,Off-OR%,Off-FTRate,AdjDE,Def-eFG%,Def-TO%,Def-OR%,Def-FTRate
0,Gonzaga,WCC,72.1,121.7,61.5,17.2,34.9,30.5,89.0,43.5,18.5,22.8,25.3
1,Purdue,B10,68.0,119.3,62.6,17.7,41.3,39.9,92.0,45.0,15.3,22.2,18.0
2,Villanova,BE,64.1,117.8,56.6,9.7,29.0,29.1,94.4,53.8,22.7,31.8,24.3
3,Baylor,B12,69.4,115.7,53.7,17.4,39.5,21.7,88.4,44.9,28.6,24.7,28.5
4,Kansas,B12,71.0,115.7,57.3,14.6,30.4,29.7,90.1,48.0,20.0,26.7,22.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,Chicago St.,WAC,68.6,85.8,47.5,21.9,26.7,24.2,112.6,56.5,17.8,24.2,38.8
368,Texas A&M-Corpus Christi,SWAC,70.4,84.8,44.0,20.2,19.8,22.4,112.0,60.5,20.0,34.4,32.4
369,Maryland-Eastern Shore,MEAC,67.6,84.7,44.1,17.9,23.7,22.3,107.1,51.0,19.9,30.0,31.7
370,Maine,AE,66.9,84.7,36.1,24.8,23.0,23.1,105.3,59.6,19.5,31.0,30.7


In [423]:
import kenpompy.misc as kp
kp.get_trends(browser).head(25)

,Season,Efficiency,Tempo,eFG%,TO%,OR%,FTRate,2P%,3P%,3PA%,FT%,A%,Blk%,Stl%,NST%,AvgHt,Cont,HomeWin%,PPG
0,2022,100.1,69.5,49.5,19.3,28.5,30.5,49.3,33.3,38.1,70.5,50.8,9.2,9.5,9.8,77.0,48.5,NaN,70.1
1,2021,101.4,69.2,50.1,18.9,28.0,31.4,49.8,33.8,37.4,71.0,51.8,8.8,9.1,9.8,76.9,46.0,57.5,70.8
2,2020,100.8,69.1,49.6,18.9,28.0,32.6,49.4,33.3,37.5,70.8,51.5,8.9,9.0,9.9,76.8,46.9,61.0,70.2
3,2019,103.2,69.0,50.7,18.5,28.4,33.0,50.1,34.4,38.7,70.7,51.9,9.3,8.9,9.7,76.8,47.8,59.0,71.9
4,2018,104.2,69.4,51.0,18.4,28.7,33.5,50.0,35.1,37.5,71.4,52.3,9.3,8.7,9.7,76.8,48.4,61.0,73.6
5,2017,103.7,69.4,50.5,18.5,29.3,35.3,49.3,35.0,36.4,70.4,52.4,9.2,8.7,9.8,76.8,49.5,59.0,72.7
6,2016,103.3,69.1,49.9,18.1,29.8,36.6,48.7,34.7,35.4,70.0,52.4,9.2,8.6,9.5,76.8,49.2,60.5,72.3
7,2015,102.1,64.9,49.0,19.1,31.0,37.0,47.8,34.3,34.2,69.3,53.1,9.6,9.4,9.7,76.7,49.4,59.8,66.9
8,2014,104.5,66.5,49.6,18.3,31.4,40.5,48.5,34.4,32.9,69.9,51.8,9.7,9.0,9.3,76.7,51.4,60.3,70.2
9,2013,100.6,65.9,48.6,20.0,31.8,35.9,47.5,33.9,33.0,69.4,53.6,9.3,9.9,10.0,76.6,50.3,61.3,66.9
